# Q4 (BAM)

## Import libraries

In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt

## BAM Network Class

In [40]:
class BamNetwork:

  def __init__(self, input_patterns, output_patterns):
    self.input_patterns = input_patterns
    self.memory = output_patterns
    self.input_nodes_count = len(self.input_patterns[0].flatten())
    self.output_nodes_count = len(self.memory[0].flatten())
    self.weights = np.zeros((self.input_nodes_count, self.output_nodes_count))
    self.weight_matrix_initialization()
    self.treshold = 0

  def weight_matrix_initialization(self): 
    for index in range(len(input_patterns)):
      input_pattern = self.input_patterns[index]
      output_pattern = self.memory[index]
      flatten_input_pattern = input_pattern.reshape((1, self.input_nodes_count))
      flatten_output_pattern = output_pattern.reshape((1, self.output_nodes_count))
      self.weights += np.dot(np.transpose(flatten_input_pattern), flatten_output_pattern)

  def predict(self, pattern, is_x=True, verbose=True):
    seen_xs = set()
    seen_ys = set()

    if is_x:
      self.x = pattern.flatten()
      self.y = np.zeros(self.memory[0].shape).flatten()
      while True:
        flatten_x = self.x.reshape((1, self.input_nodes_count))
        self.y = np.dot(flatten_x, self.weights)
        self.y[self.y > self.treshold] = 1
        self.y[self.y < self.treshold] = -1
        
        if tuple(self.y.flatten()) in seen_ys:
          if verbose:
            print('See old pattern... No memory :(')
          return self.x.reshape(self.input_patterns[0].shape), self.y.reshape(self.memory[0].shape), False
        seen_ys.add(tuple(self.y.flatten()))

        flatten_y = self.y.reshape((1, self.output_nodes_count))
        self.x = np.dot(flatten_y, self.weights.T)
        self.x[self.x > self.treshold] = 1
        self.x[self.x < self.treshold] = -1

        if tuple(self.x.flatten()) in seen_xs:
          if verbose:
            print('See old pattern... No memory :(')
          return self.x.reshape(self.input_patterns[0].shape), self.y.reshape(self.memory[0].shape), False
        seen_xs.add(tuple(self.x.flatten()))
        
        for memorise_pattern in self.memory:
          if (memorise_pattern.flatten() == self.y).all():
            for inp in self.input_patterns:
              if (inp.flatten() == self.x).all():
                if verbose:
                  print('I remember =))')
                return inp, memorise_pattern, True
    else:
      self.x = np.zeros(self.memory[0].shape).flatten()
      self.y = pattern.flatten()
      while True:
        flatten_y = self.y.reshape((1, self.output_nodes_count))
        self.x = np.dot(flatten_y, self.weights.T)
        self.x[self.x > self.treshold] = 1
        self.x[self.x < self.treshold] = -1

        if tuple(self.x.flatten()) in seen_xs:
          if verbose:
            print('See old pattern... No memory :(')
          return self.x.reshape(self.input_patterns[0].shape), self.y.reshape(self.memory[0].shape), False
        seen_xs.add(tuple(self.x.flatten()))

        flatten_x = self.x.reshape((1, self.input_nodes_count))
        self.y = np.dot(flatten_x, self.weights)
        self.y[self.y > self.treshold] = 1
        self.y[self.y < self.treshold] = -1
        
        if tuple(self.y.flatten()) in seen_ys:
          if verbose:
            print('See old pattern... No memory :(')
          return self.x.reshape(self.input_patterns[0].shape), self.y.reshape(self.memory[0].shape), False
        seen_ys.add(tuple(self.y.flatten()))
        
        for inp in self.input_patterns:
          if (inp.flatten() == self.x).all():
            for memorise_pattern in self.memory:
              if (memorise_pattern.flatten() == self.y).all():
                if verbose:
                  print('I remember =))')
                return inp, memorise_pattern, True

### Show matrix function

In [41]:
def show_result(matrix, pattern_found=None):
  if pattern_found is not None:
    if not pattern_found:
      print("Best achieved:")
    else:
      print("Result:")
  for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
      if matrix[i][j] == 1:
        print("#", end="")
      else:
        print("-", end="")
    print("")
  print("___________________________")

## Test network with A, B, C

### Print weight matrix

In [42]:
a_in_pattern = np.array([ [-1,1,-1],
                          [1,-1, 1],
                          [1, 1, 1],
                          [1,-1, 1],
                          [1,-1, 1]])
b_in_pattern = np.array([ [1, 1,-1],
                          [1,-1, 1],
                          [1, 1,-1],
                          [1,-1, 1],
                          [1, 1,-1]])
c_in_pattern = np.array([ [-1, 1, 1],
                          [ 1,-1,-1],
                          [ 1,-1,-1],
                          [ 1,-1,-1],
                          [-1, 1, 1]])

a_out_pattern = np.array([-1,-1,-1])
b_out_pattern = np.array([-1,-1, 1])
c_out_pattern = np.array([-1, 1,-1])

input_patterns = np.array([a_in_pattern, b_in_pattern, c_in_pattern])
output_patterns = np.array([a_out_pattern, b_out_pattern, c_out_pattern])
bam_net = BamNetwork(input_patterns, output_patterns)

print(bam_net.weights)

[[ 1. -1.  3.]
 [-3. -1. -1.]
 [ 1.  3. -1.]
 [-3. -1. -1.]
 [ 3.  1.  1.]
 [-1. -3.  1.]
 [-3. -1. -1.]
 [-1. -3.  1.]
 [ 1. -1. -1.]
 [-3. -1. -1.]
 [ 3.  1.  1.]
 [-1. -3.  1.]
 [-1. -3.  1.]
 [-1.  1.  1.]
 [-1.  1. -3.]]


### Test network ability

In [43]:
a_x_result, a_y_result, found = bam_net.predict(a_in_pattern)
show_result(a_x_result)
print(a_y_result)
print("*************")
b_x_result, b_y_result, found = bam_net.predict(b_in_pattern)
show_result(b_x_result)
print(b_y_result)
print("*************")
c_x_result, c_y_result, found = bam_net.predict(c_in_pattern)
show_result(c_x_result)
print(c_y_result)
print("*************")
a_x_result, a_y_result, found = bam_net.predict(a_out_pattern, is_x=False)
show_result(a_x_result)
print(a_y_result)
print("*************")
b_x_result, b_y_result, found = bam_net.predict(b_out_pattern, is_x=False)
show_result(b_x_result)
print(b_y_result)
print("*************")
c_x_result, c_y_result, found = bam_net.predict(c_out_pattern, is_x=False)
show_result(c_x_result)
print(c_y_result)
print("*************")

I remember =))
-#-
#-#
###
#-#
#-#
___________________________
[-1 -1 -1]
*************
I remember =))
##-
#-#
##-
#-#
##-
___________________________
[-1 -1  1]
*************
I remember =))
-##
#--
#--
#--
-##
___________________________
[-1  1 -1]
*************
I remember =))
-#-
#-#
###
#-#
#-#
___________________________
[-1 -1 -1]
*************
I remember =))
##-
#-#
##-
#-#
##-
___________________________
[-1 -1  1]
*************
I remember =))
-##
#--
#--
#--
-##
___________________________
[-1  1 -1]
*************


## Add noise and test network robustness

### Add noise function

In [44]:
def add_noise(pattern, noise_percentage=0.3):
  noisy_pattern = np.copy(pattern)
  pattern_cells = len(pattern.flatten())
  mistaken_cells_count = math.floor(noise_percentage * pattern_cells)
  to_change_indices = random.sample(range(pattern_cells), mistaken_cells_count)
  for index in to_change_indices:
    to_change_row = index // noisy_pattern.shape[1]
    to_change_col = index % noisy_pattern.shape[1]
    noisy_pattern[to_change_row][to_change_col] *= -1
  return noisy_pattern

### Test network

In [45]:
ten_noisy_a = add_noise(a_in_pattern, 0.1)
a_x_result, a_y_result, found = bam_net.predict(ten_noisy_a)
show_result(a_x_result)
print(a_y_result)
print("*************")
ten_noisy_b = add_noise(b_in_pattern, 0.1)
b_x_result, b_y_result, found = bam_net.predict(ten_noisy_b)
show_result(b_x_result)
print(b_y_result)
print("*************")
ten_noisy_c = add_noise(c_in_pattern, 0.1)
c_x_result, c_y_result, found = bam_net.predict(ten_noisy_c)
show_result(c_x_result)
print(c_y_result)
print("*************")

fourty_noisy_a = add_noise(a_in_pattern, 0.4)
a_x_result, a_y_result, found = bam_net.predict(fourty_noisy_a)
show_result(a_x_result)
print(a_y_result)
print("*************")
fourty_noisy_b = add_noise(b_in_pattern, 0.4)
b_x_result, b_y_result, found = bam_net.predict(fourty_noisy_b)
show_result(b_x_result)
print(b_y_result)
print("*************")
fourty_noisy_c = add_noise(c_in_pattern, 0.4)
c_x_result, c_y_result, found = bam_net.predict(fourty_noisy_c)
show_result(c_x_result)
print(c_y_result)
print("*************")

I remember =))
-#-
#-#
###
#-#
#-#
___________________________
[-1 -1 -1]
*************
I remember =))
##-
#-#
##-
#-#
##-
___________________________
[-1 -1  1]
*************
I remember =))
-##
#--
#--
#--
-##
___________________________
[-1  1 -1]
*************
I remember =))
-#-
#-#
###
#-#
#-#
___________________________
[-1 -1 -1]
*************
I remember =))
-#-
#-#
###
#-#
#-#
___________________________
[-1 -1 -1]
*************
I remember =))
-#-
#-#
###
#-#
#-#
___________________________
[-1 -1 -1]
*************


In [48]:
a_correct = 0
b_correct = 0
c_correct = 0

for _ in range(100):
  ten_noisy_a = add_noise(a_in_pattern, 0.1)
  a_x_result, a_y_result, found = bam_net.predict(ten_noisy_a, verbose=False)
  if (a_y_result == a_out_pattern).all():
    a_correct += 1

  ten_noisy_b = add_noise(b_in_pattern, 0.1)
  b_x_result, b_y_result, found = bam_net.predict(ten_noisy_b, verbose=False)
  if (b_y_result == b_out_pattern).all():
    b_correct += 1

  ten_noisy_c = add_noise(c_in_pattern, 0.1)
  c_x_result, c_y_result, found = bam_net.predict(ten_noisy_c, verbose=False)
  if (c_y_result == c_out_pattern).all():
    c_correct += 1
print("Ten percent noise accuracy:")
print("A: {} - B: {} - C: {}".format(a_correct, b_correct, c_correct))

Ten percent noise accuracy:
A: 100 - B: 100 - C: 100


## Test (0, -1, -1) output

In [50]:
pattern = np.array([0, -1, -1])
x_result, y_result, found = bam_net.predict(pattern, is_x=False)
show_result(x_result)
print(y_result)
print("*************")

I remember =))
-#-
#-#
###
#-#
#-#
___________________________
[-1 -1 -1]
*************


## Other charcters test

In [54]:
d_in_pattern = np.array([ [1, 1,-1],
                          [1,-1, 1],
                          [1,-1, 1],
                          [1,-1, 1],
                          [1, 1,-1]])
e_in_pattern = np.array([ [1, 1, 1],
                          [1,-1,-1],
                          [1, 1,-1],
                          [1,-1,-1],
                          [1, 1, 1]])
f_in_pattern = np.array([ [1, 1, 1],
                          [1,-1,-1],
                          [1, 1,-1],
                          [1,-1,-1],
                          [1,-1,-1]])
g_in_pattern = np.array([ [-1, 1, 1],
                          [ 1,-1,-1],
                          [ 1,-1, 1],
                          [ 1,-1, 1],
                          [-1, 1, 1]])
h_in_pattern = np.array([ [ 1,-1, 1],
                          [ 1,-1, 1],
                          [ 1, 1, 1],
                          [ 1,-1, 1],
                          [ 1,-1, 1]])

d_out_pattern = np.array([-1, 1, 1])
e_out_pattern = np.array([ 1,-1,-1])
f_out_pattern = np.array([ 1,-1, 1])
g_out_pattern = np.array([ 1, 1,-1])
h_out_pattern = np.array([ 1, 1, 1])

input_patterns = np.array([a_in_pattern, b_in_pattern, c_in_pattern, d_in_pattern, e_in_pattern, f_in_pattern, g_in_pattern, h_in_pattern])
output_patterns = np.array([a_out_pattern, b_out_pattern, c_out_pattern, d_out_pattern, e_out_pattern, f_out_pattern, g_out_pattern, h_out_pattern])
bam_net2 = BamNetwork(input_patterns, output_patterns)

print(bam_net2.weights)

[[ 2. -2.  6.]
 [-2. -2. -2.]
 [ 6.  2. -2.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [-4.  0.  4.]
 [ 0.  0.  0.]
 [ 2. -6.  2.]
 [ 0.  4.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [-2.  2.  2.]
 [ 0. -4.  4.]
 [-2.  2. -2.]
 [ 2.  2. -6.]]


## Test extended network

In [55]:
a_x_result, a_y_result, found = bam_net2.predict(a_in_pattern)
show_result(a_x_result)
print(a_y_result)
print("*************")
b_x_result, b_y_result, found = bam_net2.predict(b_in_pattern)
show_result(b_x_result)
print(b_y_result)
print("*************")
c_x_result, c_y_result, found = bam_net2.predict(c_in_pattern)
show_result(c_x_result)
print(c_y_result)
print("*************")
a_x_result, a_y_result, found = bam_net2.predict(a_out_pattern, is_x=False)
show_result(a_x_result)
print(a_y_result)
print("*************")
b_x_result, b_y_result, found = bam_net2.predict(b_out_pattern, is_x=False)
show_result(b_x_result)
print(b_y_result)
print("*************")
c_x_result, c_y_result, found = bam_net2.predict(c_out_pattern, is_x=False)
show_result(c_x_result)
print(c_y_result)
print("*************")

See old pattern... No memory :(
##-
--#
-#-
--#
#--
___________________________
[-1. -1.  1.]
*************
See old pattern... No memory :(
##-
--#
-#-
--#
#--
___________________________
[-1. -1.  1.]
*************
See old pattern... No memory :(
--#
---
--#
---
-##
___________________________
[ 1.  1. -1.]
*************
See old pattern... No memory :(
-#-
---
-#-
---
-##
___________________________
[-1. -1. -1.]
*************
See old pattern... No memory :(
##-
--#
-#-
--#
#--
___________________________
[-1. -1.  1.]
*************
See old pattern... No memory :(
-#-
---
--#
--#
-##
___________________________
[-1.  1. -1.]
*************


## Calculate hamming distances

### Hamming distance funciton

In [56]:
def hamming_distance(pattern_1, pattern_2):
  return np.count_nonzero(pattern_1 != pattern_2)

### See results

In [60]:
words = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
print("Hamming distances:")
for i in range(len(input_patterns)):
  for j in range(i+1, len(input_patterns)):
    pattern_1 = input_patterns[i]
    pattern_2 = input_patterns[j]
    hamming_d = hamming_distance(pattern_1, pattern_2)
    print("{} - {}: {}".format(words[i], words[j], hamming_d))

Hamming distances:
A - B: 4
A - C: 7
A - D: 4
A - E: 6
A - F: 6
A - G: 5
A - H: 3
B - C: 7
B - D: 2
B - E: 4
B - F: 4
B - G: 7
B - H: 5
C - D: 7
C - E: 3
C - F: 5
C - G: 2
C - H: 8
D - E: 6
D - F: 6
D - G: 5
D - H: 5
E - F: 2
E - G: 5
E - H: 5
F - G: 7
F - H: 5
G - H: 6
